<div class="alert alert-block alert-info">
     <b>Import Modules Needed</b> <i>(need pandas >= 1.2 and python >= 3.9.0 for all the data cleaning techniques to work)</i>
</div>

In [ ]:
import pandas as  pd
import os
import sweetviz as sv
import numpy as np
import dtale
import cudf
import dask_cudf
from pandasql import sqldf

#!pip install git+https://github.com/innovationOUtside/fstring-magic.git

In [ ]:
#%load_ext fstring_magic

<div class="alert alert-block alert-info">
    <b>Jupyter Plumbing</b>
</div>


In [ ]:
# WINDOWS - get IP address of running Ubuntu instance
# Run an admin elevated powershell instance (not ubuntu VM)
# netsh interface portproxy add v4tov4 listenport=40000 listenaddress=0.0.0.0 connectport=40000 connectaddress=172.21.240.230
# This will open up a port that the dtale package between local windows machine and the Ubuntu VM need to display correctly
#!ifconfig
# netsh interface portproxy add v4tov4 listenport=40000 listenaddress=0.0.0.0 connectport=40000 connectaddress=172.21.240.230


<div class="alert alert-block alert-info">
    <b>Check Dtale port</b>
</div

In [ ]:
dtale.instances()

In [ ]:
dtale.show(pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]}))


In [ ]:
# %load C:\users\derek\.jupyter\startup.py
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', 100)

pd.set_option('display.max_colwidth', None)



pd.set_option("display.max_rows", None)

pd.options.display.max_columns = 500

pd.options.display.max_seq_items = 500



<div class="alert alert-block alert-info">
    <b>Find CSV Files in working dir</b>
</div>

In [ ]:
csvStore = os.path.join(os.getcwd(),'__csvStore' )
csvlist = []
for file in enumerate(os.listdir(csvStore)):
    if file[1].endswith(".csv"):
        csvlist.append(os.path.join(str(csvStore), file[1]))
        print(os.path.join(str(csvStore), file[1]))
        

<div class="alert alert-block alert-info">
    <b>Import the raw data</b>
</div>

I learned through manual trial and error many different data issues when trying to import the csv <br>
The main issue was with datetimes, and columns recognized as containing mixed data types. <br>
Some columns can be converted during import but other must be coerced after <br>



In [ ]:
csvfile = 'orig_data2.csv'
list(enumerate(csvlist))
csvlist[5]

In [ ]:
df = pd.read_csv(csvlist[5], na_values = ['IGNORED', '-', 'UNKNOWN', 'UK - OTHER UNKNOWN', 'UK - UNKNOWN'], dtype = {'ceas_ascode': object, 'M1322_NBR_PRSULC_STG1':object})
len(df)

In [ ]:
%%fstring
__There are {len(df.columns)} columns, in the df__

<div class="alert alert-block alert-success">
    <b>Create SweetViz EDA Report</b>
</div>

In [ ]:
#analyzing the dataset
# hospice_death_report = sv.analyze(df, pairwise_analysis = 'off')
#display the report
# hospice_death_report.show_html('cleaned_hospice_dead.html')


# Part 1 Cleanup of full dataset

<div class="alert alert-danger" role="alert">
    <b>Manually Clean up Data (remove cols) based on SweetViz report</b>
</div>

In [ ]:
df_clean = df.drop(df.columns[np.r_[3, 5, 14, 15, 36, 47:70, 87:94, 100:104, 105:107, 122:129, 136, 138:149, 151, 168, 178:184, 192:194, 195:198, 201, 204:211, 212:218, 219, 221:223, 224:229, 230:232, 240:261, 261:266]], axis = 1)
                                    
#df_clean.columns


# REMOVE COLS WITH ONLY NAN
df = df.dropna(axis=1, how='all')




In [ ]:
%%fstring
__There are {len(df.columns)} columns remaining if we simply eliminate columns with 100% null values, or {len(df_clean.columns)} based on my manual removal process__

<div class="alert alert-danger" role="alert">
    <b>Compare manual elimination of empty columns via SweetViz to pandas automatic removal</b>
</div>

In [ ]:
(df.columns).difference(df_clean.columns)

In [ ]:
df = df.drop((df.columns).difference(df_clean.columns), axis = 1)
len(df.columns)
len(df.columns) == len(df_clean.columns)

In [ ]:
%%fstring
__We can remove these columns even though they are not strictly NAN. They are majority NAN and are not useful to the analysis__

<div class="alert alert-block alert-info">
    <b>Find Columns with Dates and convert them to datetime type</b>
</div>

In [ ]:
cols_w_dates = df.columns[(df.columns.str.contains('DT', case=False) | df.columns.str.contains('date', case=False) | df.columns.str.contains('OFEPISODE', case=False))] 
df[cols_w_dates] = df[cols_w_dates].astype('datetime64[s]')
cols_w_dates

<div class="alert alert-block alert-info">
    <b>Find Columns that contain mixed data types to investigate for possible data quality issues</b>
</div>


In [ ]:
if mixed_dtypes := {c: dtype for c in df.columns if (dtype := pd.api.types.infer_dtype(df[c])).startswith("mixed")}:
    raise TypeError(f"Dataframe has one more mixed dtypes: {mixed_dtypes}")

In [ ]:
# If Run in Linux pd.read_csv was detecting mixed datatype differently than windows

In [ ]:
# # Coerce bad columns to int dtype
# for col in ['M1311_NSTG_CVRG_SOCROC_E2','M1322_NBR_PRSULC_STG1', 'M2200_THER_NEED_NBR']:
#   df[col] = pd.to_numeric(df[col], errors='coerce')
         
# for col in ['M1322_NBR_PRSULC_STG1']:
#    df[col] = pd.to_numeric(df[col], errors='coerce')


In [ ]:
# Descriptive IDs
df.columns[:26]
colsMeta = df.columns[:26]
# OASIS Factors
df.columns[26:]
colsOASIS = df.columns[26:]

<div class="alert alert-info" role="alert">
    <b>Percentage of Null Values by Column to see if there are errors or we can eliminate unhelpful columns</b>
</div>


In [ ]:
(df.loc[:,colsOASIS].isnull().sum()/len(df.loc[:,colsOASIS])).sort_values(ascending=False)

> Since we previously eliminated completely empty columns we should not expect to see empty columns here. We see the columns ranked by degree of missing values <BR>
  Prime candidates for elimination are columns with 90% or greater nulls. We need both training and testing sets to be populated with the same metrics or there is no apples-apples comparison



# Part 2 Cleanup of Alive vs Dead

After cleaning up the dataset as a whole, we should dive deeper and cleanup based on the two main types of patients: Alive vs Dead which end up being training and testing sets.  We need a common set of features between both of these groups or we wont have an Apples to Apples comparison

<mark>The dataset contains 3 Patient Groups (Disposition ID): Died in Hospice, Died in Home Health Care, Alive so we now need to move to the 2nd stage of EDA and that is getting a clean deadpatients list</mark>



<div class="alert alert-info" role="alert">
    <b>% of null values by Disposition ID</b>
</div>

# Missing Values (Pct Total) for Disposition 3 Patients (Hospice)

In [ ]:
(df.loc[:,colsOASIS][df['DispositionId']==3].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==3])).sort_values(ascending=False)
mask3 = (df.loc[:,colsOASIS][df['DispositionId']==3].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==3])>=.75).sort_values(ascending=False)
#df.loc[:, ~mask]

In [ ]:
df.loc[:,colsOASIS].loc[:, mask3].columns
mask3_sr = df.loc[:,colsOASIS].loc[:, mask3].columns

# Missing Values (Pct Total) for Disposition 2 Patients (HHG)

In [ ]:
(df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])).sort_values(ascending=False)
mask2 = (df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])>=.75).sort_values(ascending=False)

In [ ]:
df.loc[:,colsOASIS].loc[:, mask2].columns
mask2_sr = df.loc[:,colsOASIS].loc[:, mask2].columns

# Features Missing All Values for All Dead Patients (Disposition 2 and 3 combined) - We can Eliminate from df

In [ ]:
len(mask2_sr.union(mask3_sr))
mask2_sr.union(mask3_sr)

____

# There are no fields completley null for the alive patients cohort except there ae some close

In [ ]:
(df.loc[:,colsOASIS][df['DispositionId']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==1])).sort_values(ascending=False)
mask1 = (df.loc[:,colsOASIS][df['DispositionId']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==1])>=.75).sort_values(ascending=False)

In [ ]:
bigmask = (mask2_sr.union(mask3_sr)).union(df.loc[:,colsOASIS].loc[:, mask1].columns)
bigmask

In [ ]:
(df.loc[:,colsOASIS][(df['DispositionId']==2) | (df['DispositionId']==3) ].isnull().sum()/len(df.loc[:,colsOASIS][(df['DispositionId']==2) + (df['DispositionId']==3)])).sort_values(ascending=False)

# Depending on the formula we can wither see True = Column is above threshold, or True if Column is below

In [ ]:
fullalive = (1-(df.loc[:,colsOASIS][df['DispositionId']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==1]))>=.90).sort_values(ascending=False)
emptyalive = (df.loc[:,colsOASIS][df['DispositionId']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==1])>=.90).sort_values(ascending=False)
emptyalive
maskfilter1 = emptyalive

In [ ]:
fulldead = ((1-(df.loc[:,colsOASIS][(df['DispositionId']==2) | (df['DispositionId']==3) ].isnull().sum())/len(df.loc[:,colsOASIS][(df['DispositionId']==2) + (df['DispositionId']==3)]))>=.90).sort_values(ascending=False)
emptydead = (((df.loc[:,colsOASIS][(df['DispositionId']==2) | (df['DispositionId']==3) ].isnull().sum())/len(df.loc[:,colsOASIS][(df['DispositionId']==2) + (df['DispositionId']==3)]))>=.90).sort_values(ascending=False)
emptydead
maskfilter23 = emptydead

In [ ]:
len(df.loc[:,colsOASIS].loc[:, maskfilter1].columns)
(df.loc[:,colsOASIS].loc[:, maskfilter1].columns)
len(df.loc[:,colsOASIS].loc[:, maskfilter23].columns)
(df.loc[:,colsOASIS].loc[:, maskfilter23].columns)

In [ ]:
len((df.loc[:,colsOASIS].loc[:, maskfilter1].columns).intersection(df.loc[:,colsOASIS].loc[:, maskfilter23].columns))
(df.loc[:,colsOASIS].loc[:, maskfilter1].columns).intersection(df.loc[:,colsOASIS].loc[:, maskfilter23].columns)

<div class="alert alert-danger" role="alert">
    <b>Dropping Columns that are NULL for Dead Patients</b>
</div>

<div class="alert alert-danger" role="alert">
    <b>Dropping Columns with too many null values across Dead Patients (Dispo 2 and Dispo 3)</b>
</div>

In [ ]:
finaldrop = (df.loc[:,colsOASIS].loc[:, maskfilter23].columns)[:-1].union(bigmask)
len(finaldrop)


In [ ]:
df = df.drop(finaldrop, axis=1)

In [ ]:
len(df.columns)

# Part 2.1 Remove Duplicate Rows

<div class="alert alert-block alert-info">
    <b>Rearrange Columns to a more preferable order</b>
</div>

In [ ]:
cols_w_dates = cols_w_dates.union(df.columns[(df.columns.str.contains('DAYS', case=False))])
cols_w_dates = ['epi_SocDate', 'epi_StartOfEpisode', 'epi_EndOfEpisode', 'epi_DischargeDate', 'M0906_DC_TRAN_DTH_DT', 'M0090_INFO_COMPLETED_DT', 'cedd_dateofdeath', 'DaysToDeath']
firstcols = [ 'DispositionId', 'pa_id', 'year_born', 'pa_gender', 'epi_id', 'ceo_id']
first_cols = firstcols+cols_w_dates
move_columns = first_cols + (df.columns.drop(first_cols).tolist())
df = df[move_columns]

<div class="alert alert-success" role="alert">
    <b>Create Filters that remove the primary keys</b>
</div>

In [ ]:
dfcols_noHIPPS = df.columns.difference(['ceo_HIPPS', 'ceo_HHRG', 'ceo_id'])


<div class="alert alert-info" role="alert">
    <b>Remove duplicates while keeping rows that have HIPPS codes</b>
</div>

#### Sort the Data by CEO_HIPPS with NANS at the bottom, so that when we drop duplicates we keep the first instance with the HIPPS

In [ ]:
uniq_indx = (df.sort_values(by="ceo_HIPPS", na_position='last').drop_duplicates(subset=dfcols_noHIPPS, keep='first')).index

#### Creates an index which is the row numbers we want to keep

In [ ]:
df = df.loc[uniq_indx]
len(uniq_indx)

In [ ]:
dtale.show(df.head(100))

## 2.2 Fix Divergent pa_id


<div class="alert alert-info" role="alert">
    <b>Different Patient Id's for the same patient that transferred into Hospice</b>
</div>

In [ ]:
# This will return a boolean series indicating which rows are duplicated and keep both

In [ ]:
t = df['ceo_id'].duplicated(keep=False)
len(t)

In [ ]:
dtale.show(df[t])

In [ ]:
df.sort_values(by=['ceo_id', 'pa_id'], ascending=[True, True], inplace=True)
s = ~df.ceo_id.duplicated(keep='first')
df['pa_id'] = df['ceo_id'].map(df[s].set_index('ceo_id')['pa_id'])
#df['DaysToDeath'] = df['ceo_id'].map(df[s].set_index('ceo_id')['DaysToDeath'])
#df['cedd_dateofdeath'] = df['ceo_id'].map(df[s].set_index('ceo_id')['cedd_dateofdeath'])

# s = ~deaddupes2.ceo_id.duplicated()
# deaddupes2['pa_id'] = deaddupes2.ceo_id.map(deaddupes2[s].set_index('ceo_id')['pa_id'])


In [ ]:
dtale.show(df[s])


<div class="alert alert-info" role="alert">
    <b>Verification that both ceo_id now are correctly labled with one pa_id</b>
</div>

In [ ]:
dtale.show(df[df['ceo_id'] == 140968])

## Fix Duplicate Disposition ID

# This group of patients has more than 1 Disposition ID which was discovered when counting the number of unique patient id's in the entire dataset vs the number of unique patients with each disposition

In [ ]:
# Many of these will be the same from above

In [ ]:
grp = df.groupby('pa_id').nunique()['DispositionId'].apply(lambda g: g>1)
len(grp[grp].index)
grp[grp].index
dupedispo = df[df['pa_id'].isin(grp[grp].index)].reset_index(drop=True)
dupedispo.to_csv(csvStore+'/duplicate_DispositionIDs.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
dtale.show(dupedispo.reset_index(drop=True).sort_values(by=['pa_id', 'DispositionId']))

In [ ]:
df.sort_values(by=['pa_id', 'DispositionId'], ascending=[True, True], inplace=True)
d = ~df.pa_id.duplicated(keep='last')
df['DispositionId'] = df['pa_id'].map(df[d].set_index('pa_id')['DispositionId'])
df['DaysToDeath'] = df['pa_id'].map(df[d].set_index('pa_id')['DaysToDeath'])
df['cedd_dateofdeath'] = df['pa_id'].map(df[d].set_index('pa_id')['cedd_dateofdeath'])

In [ ]:
dtale.show(df[df.pa_id==56831])

<div class="alert alert-block alert-success">
    <b>Export Cleaned DataSet</b>
</div>

In [ ]:
df.to_csv((csvStore+'/cleaned_data.csv'), sep=',', encoding='utf-8', index=False)

<div class="alert alert-block alert-success">
    <b>EDF for our final Factor List</b>
</div>

In [ ]:
#analyzing the dataset
high_level_cleaned = sv.analyze(df, pairwise_analysis = 'off')
#display the report
high_level_cleaned .show_html('high_level_cleaned.html')


<div class="alert alert-danger">
  <strong>Create a GPU version of DFF for faster processing</strong> 
</div>

In [ ]:
dff = cudf.DataFrame(df)
len(dff)

<div class="alert alert-warning" role="alert">
    <b>Tools to Filter DF and understand the dataset</b>
</div>


<div class="alert alert-warning" role="alert">
    <b>Enumerate the columns to help to more quickly ID them</b>
</div>


In [ ]:
column_list = pd.DataFrame(enumerate(df.columns.values.tolist())).set_index(0)
column_list

In [ ]:
df.dtypes

<div class="alert alert-warning" role="alert">
    <b> Filter Tool to find contents of specific cells</b>
</div>

In [ ]:
df.iloc[13,107] # add rows if i want to find a specific cell
df.iloc[13:14,:10] # add rows if i want to find a specific cell

<div class="alert alert-warning" role="alert">
    <b>Eliminate Columns with only 1 distinct value</b>
</div>


In [ ]:
cols=[]
for col in df.columns:
    if len(df[col].unique()) >= 1 and len(df[col].unique()) <= 3:
        cols.append(col)
len(cols)

In [ ]:
cols

In [ ]:
df.nunique().sort_values(ascending=False)

The number of unique values in a column provide insight into the nature of the column.<br>
The OASIS data is standardized so there are only a few responses the nurses must choose.<br>
pa_id will have many more unique value, followed by episode id, all the way to a primary key with a unique value for each row<br>
<br>
We Eliminated Columns From Dead Patients if all the values were missing.<br>
There is no Point in keeping these columns for the independent variables either

# Part 3 - Get Patient Lists

In [ ]:
df.groupby('DispositionId').agg({'pa_id': 'nunique'})

> Within each disposition their are a minority of patients who have SOC dates, earlier than their first episode.  Only within the alive patients do we have individual patients with both date types

<div class="alert alert-danger" role="alert">
    <b>Get our list of Dead Patients</b>
</div>

In [ ]:

qless_than_1yr = '''SELECT pa_id, DaysToDeath, max(JULIANDAY(epi_DischargeDate)) - min(JULIANDAY(epi_SocDate)) as DateDiff
           FROM df
           WHERE DispositionId != 1
           GROUP BY pa_id
           HAVING (DateDiff <= 365 AND DaysToDeath <= 365) OR (DateDiff <= 365 AND DaysToDeath IS NULL) OR (DateDiff > 365 AND DaysToDeath <= 365) OR (DateDiff IS NULL AND DaysToDeath <= 365)'''

In [ ]:
sql_less_than_1yr = sqldf(qless_than_1yr)
deadset = set(sql_less_than_1yr.pa_id)

In [ ]:
len(deadset) #WHERE (dcc_desc = 'PATIENT EXPIRED' AND dcr_desc = 'PATIENT EXPIRED')

In [ ]:
dtale.show(sql_less_than_1yr)

In [ ]:
df_dead = df[df.pa_id.isin(deadset)]
f"There are {len(deadset)}, patients who died in less than 1 year"

In [ ]:
df_dead.groupby('DispositionId').agg({'pa_id': 'nunique'})
dtale.show(df_dead)

<div class="alert alert-success" role="alert">
    <b>Export Dataset with Bad Columns Removed and Dead Patients with more than 365 days removed</b>
</div>

In [ ]:
df_dead.to_csv(csvStore+'/dead_patients_final.csv', sep=',', encoding='utf-8', index=False)

<div class="alert alert-danger" role="alert">
    <b>Get our list of Patients who lived longer than 1 year (alive)</b>
</div>

# Get Patients who have lived longer than 1 year

In [ ]:

qmore_than_1yr = '''SELECT pa_id, max(JULIANDAY(epi_DischargeDate)) - min(JULIANDAY(epi_SocDate)) as DateDiff
           FROM df
           GROUP BY pa_id
           HAVING (DateDiff > 365 AND DaysToDeath > 365) OR (DateDiff > 365 AND DaysToDeath IS NULL) OR (DateDiff < 365 AND DaysToDeath > 365) OR  (DateDiff IS NULL AND DaysToDeath > 365)'''

sql_more_than_1yr = sqldf(qmore_than_1yr)



In [ ]:

aliveset = set(sql_more_than_1yr.pa_id)
df_alive = df[df.pa_id.isin(aliveset)]
f"There are only {len(aliveset)}, patients who have lived longer than 1 year"

In [ ]:
dupes = deadset.intersection(aliveset)

In [ ]:
dupes

In [ ]:
df[df.pa_id.isin(dupes)]

In [ ]:
df_alive.groupby('DispositionId').agg({'pa_id': 'nunique'})
dtale.show(df_alive)

<div class="alert alert-block alert-success">
    <b>Export Alive List</b>
</div>

In [ ]:
df_alive.to_csv(csvStore+'/alive_patients_final.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
neither = deadset.union(aliveset)    
conditions = [df['pa_id'].isin(aliveset), df['pa_id'].isin(deadset), (~df['pa_id'].isin(neither))]
values = ['0', '1', None]
df.insert(2, "mortality", np.select(conditions, values))                                                           

In [ ]:
dtale.show(df.head(1000))

In [ ]:
df.groupby('mortality').agg({'pa_id': 'nunique'})

In [ ]:
dtale.show(df[df['mortality'].isnull()])


<div class="alert alert-info" role="alert">
    <b>Final Data Sets</b>
</div>

In [ ]:
df_final = df[~df['mortality'].isnull()]
df_eliminated = df[df['mortality'].isnull()]
df_final.to_csv(csvStore+'/final.csv', sep=',', encoding='utf-8', index=False)
df_eliminated.to_csv(csvStore+'/eliminated.csv', sep=',', encoding='utf-8', index=False)